# TimesNet 中文教程
**环境配置说明：** 本Notebook为`TimesNet`支持的学习任务提供中文教程。

`TimesNet` 支持5大类任务，分别为：长期预测、短期预测、数据插补、异常检测、分类。

### 1. 安装Python 3.8。推荐执行如下命令。

In [ ]:
pip install -r ../requirements.txt

### 2. 导入依赖包

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.fft

from layers.Embed import DataEmbedding
from layers.Conv_Blocks import Inception_Block_V1   # 用于2D时序数据卷积的模块，可更换

### 3. TimesBlock 构建
`TimesNet`的核心思想在于`TimesBlock`的构建。其主要通过对数据进行FFT获取基频，然后将时间序列根据主基频分别重塑为2D变化，接着进行2D卷积，最后加权回原序列得到输出。

下面详细介绍`TimesBlock`的实现。

TimesBlock包含两个主要成员。

In [ ]:
class TimesBlock(nn.Module):
    def __init__(self, configs):
        ...
    
    def forward(self, x):
        ...

首先关注`__init__(self, configs)`的实现：

In [ ]:
def __init__(self, configs):    # configs为TimesBlock的配置
    super(TimesBlock, self).__init__()
    self.seq_len = configs.seq_len   # 序列长度
    self.pred_len = configs.pred_len # 预测长度
    self.k = configs.top_k    # 选取的主频数量
    # 参数高效设计
    self.conv = nn.Sequential(
        Inception_Block_V1(configs.d_model, configs.d_ff, num_kernels=configs.num_kernels),
        nn.GELU(),
        Inception_Block_V1(configs.d_ff, configs.d_model, num_kernels=configs.num_kernels)
    )

接下来，关注`forward(self, x)`的实现：

In [ ]:
def forward(self, x):
    B, T, N = x.size()  # B:批大小 T:序列长度 N:特征数
    period_list, period_weight = FFT_for_Period(x, self.k)
    res = []
    for i in range(self.k):
        period = period_list[i]
        if (self.seq_len + self.pred_len) % period != 0:
            length = (((self.seq_len + self.pred_len) // period) + 1) * period
            padding = torch.zeros([x.shape[0], (length - (self.seq_len + self.pred_len)), x.shape[2]]).to(x.device)
            out = torch.cat([x, padding], dim=1)
        else:
            length = (self.seq_len + self.pred_len)
            out = x
        out = out.reshape(B, length // period, period, N).permute(0, 3, 1, 2).contiguous()
        out = self.conv(out)
        out = out.permute(0, 2, 3, 1).reshape(B, -1, N)
        res.append(out[:, :(self.seq_len + self.pred_len), :])
    res = torch.stack(res, dim=-1)
    period_weight = F.softmax(period_weight, dim=1)
    period_weight = period_weight.unsqueeze(1).unsqueeze(1).repeat(1, T, N, 1)
    res = torch.sum(res * period_weight, -1)
    res = res + x
    return res

上述`FFT_for_Period`函数定义如下：

In [ ]:
def FFT_for_Period(x, k=2):
    xf = torch.fft.rfft(x, dim=1)
    frequency_list = abs(xf).mean(0).mean(-1)
    frequency_list[0] = 0
    _, top_list = torch.topk(frequency_list, k)
    top_list = top_list.detach().cpu().numpy()
    period = x.shape[1] // top_list
    return period, abs(xf).mean(-1)[:, top_list]

更直观的理解可参考下图：

![FFT 示意图](./tutorial/fft.png)

![2D 卷积示意图](./tutorial/conv.png)


更多细节可参考我们的论文：
(链接: https://openreview.net/pdf?id=ju_Uqw384Oq)

### 4. TimesNet整体结构

有了`TimesBlock`，我们可以构建`TimesNet`，它擅长提取时序数据的周期性信息，支持多种任务。

下面介绍`TimesNet`的整体结构和多任务能力。

In [ ]:
class Model(nn.Module):
    def __init__(self, configs):
        ...
    def forecast(self, x_enc, x_mark_enc, x_dec, x_mark_dec):
        ...
    def imputation(self, x_enc, x_mark_enc, x_dec, x_mark_dec, mask):
        ...
    def anomaly_detection(self, x_enc):
        ...
    def classification(self, x_enc, x_mark_enc):
        ...
    def forward(self, x_enc, x_mark_enc, x_dec, x_mark_dec, mask=None):
        ...

首先关注`__init__(self, configs)`的实现：

In [ ]:
def __init__(self, configs):
    super(Model, self).__init__()
    self.configs = configs
    self.task_name = configs.task_name
    self.seq_len = configs.seq_len
    self.label_len = configs.label_len
    self.pred_len = configs.pred_len
    self.model = nn.ModuleList([TimesBlock(configs) for _ in range(configs.e_layers)])
    self.enc_embedding = DataEmbedding(configs.enc_in, configs.d_model, configs.embed, configs.freq, configs.dropout)
    self.layer = configs.e_layers
    self.layer_norm = nn.LayerNorm(configs.d_model)
    if self.task_name == 'long_term_forecast' or self.task_name == 'short_term_forecast':
        self.predict_linear = nn.Linear(self.seq_len, self.pred_len + self.seq_len)
        self.projection = nn.Linear(configs.d_model, configs.c_out, bias=True)
    if self.task_name == 'imputation' or self.task_name == 'anomaly_detection':
        self.projection = nn.Linear(configs.d_model, configs.c_out, bias=True)
    if self.task_name == 'classification':
        self.act = F.gelu
        self.dropout = nn.Dropout(configs.dropout)
        self.projection = nn.Linear(configs.d_model * configs.seq_len, configs.num_class)

#### 4.1 预测任务
预测的基本思想是将已知序列扩展到(seq_len+pred_len)长度，通过多层TimesBlock和归一化提取周期信息，最后投影到输出空间。